In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r"../..")


from torch import nn
from torch.nn import functional as F
import torch.optim as optim
import torch
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

from callbacks import state_to_features, ACTION_MAP, ACTION_MAP_INV
from networks import AgentNet

from events import *

In [2]:
EVENT_HORIZON = {
    MOVED_LEFT: 1,
    MOVED_RIGHT: 1,
    MOVED_UP: 1,
    MOVED_DOWN: 1,
    WAITED: 2,
    INVALID_ACTION: 1,
    BOMB_DROPPED: 2,
    BOMB_EXPLODED: 2,
    CRATE_DESTROYED: 8,
    COIN_FOUND: 3,
    COIN_COLLECTED: 5,
    KILLED_OPPONENT: 8,
    KILLED_SELF: 3,
    GOT_KILLED: 3,
    OPPONENT_ELIMINATED: 0,
    SURVIVED_ROUND: 8
}
"""
REWARDS = {
    MOVED_LEFT: 1,
    MOVED_RIGHT: 1,
    MOVED_UP: 1,
    MOVED_DOWN: 1,
    WAITED: 0,
    INVALID_ACTION: -5,
    BOMB_DROPPED: 1,
    BOMB_EXPLODED: 0,
    CRATE_DESTROYED: 4,
    COIN_FOUND: 0,
    COIN_COLLECTED: 5,
    KILLED_OPPONENT: 5,
    KILLED_SELF: -8,
    GOT_KILLED: -8,
    OPPONENT_ELIMINATED: 0,
    SURVIVED_ROUND: 0
}
"""
REWARDS = {
    MOVED_LEFT: 100,
    MOVED_RIGHT: 100,
    MOVED_UP: 100,
    MOVED_DOWN: 100,
    WAITED: -100,
    INVALID_ACTION: -10,
    BOMB_DROPPED: 0,
    BOMB_EXPLODED: 0,
    CRATE_DESTROYED: 0,
    COIN_FOUND: 0,
    COIN_COLLECTED: 0,
    KILLED_OPPONENT: 0,
    KILLED_SELF: 0,
    GOT_KILLED: 0,
    OPPONENT_ELIMINATED: 0,
    SURVIVED_ROUND: 0
}

# REWARDS = {k: v + 10 for k, v in REWARDS.items()}



In [3]:
from torch.utils.data import Dataset


class BombermanDataset(Dataset):
    def __init__(self, states_dir, max_game_step=200, min_reward=1):
        self.features = []
        self.actions = []
        self.cum_rewards = []
        self.total_number = 0
        for states_file in os.listdir(states_dir):

            if not states_file.endswith('.pickle'):
                continue
            with open(os.path.join(states_dir, states_file), "rb") as f:
                data = pickle.load(f)
            self.total_number += len(data['game_state'])
            last_round = -1
            running_horizons = []
            running_rewards = []
            for i in range(len(data['game_state'])-1,-1,-1):
                game_state = data['game_state'][i]               

                    
                if last_round != game_state['round']:
                    last_round = game_state['round']
                    running_horizons = []
                    running_rewards = []
                if game_state['step'] > max_game_step:
                    continue
                    
                # determine if action should be included in training set
                
                horizons = [EVENT_HORIZON[e] for e in data['events'][i]]
                rewards = [REWARDS[e] for e in data['events'][i]]
                cr = np.where(data['events'] == "CRATE_DESTROYED")
                #print(cr)
                #rewards[np.array(cr[-1:], dtype = "int")] == 0
                #for j in cr[1:]:
                #    rewards[j] = 0             

                running_rewards = [r for j,r in enumerate(running_rewards) if running_horizons[j]>1]
                running_horizons = [h-1 for h in running_horizons if h > 1]
                running_rewards.extend(rewards)
                running_horizons.extend(horizons)
                #if "INVALID_ACTION" in data['events'][i]:
                #    continue   
                self.cum_rewards.append(np.sum(running_rewards))

                last_action = None if game_state['step'] == 1 else data['action'][i-1]
                #print("LAST ACTION:")
                #print(last_action)
                
                channels, features, act_map = state_to_features(game_state, last_action=last_action)
                action = ACTION_MAP[act_map[data['action'][i]]]
                self.features.append((
                    torch.tensor(channels[np.newaxis], dtype=torch.float),
                    torch.tensor(features, dtype=torch.float)
                ))
                self.actions.append(action)
                
                    
        self.cum_rewards = torch.tensor(self.cum_rewards, dtype=torch.float)
        self.n = np.array([np.sum([t == i for t in self.actions]) for i in range(6)])
        self.weights = np.array([1/self.n[t] for t in self.actions])
        self.proportion_selected = len(self.actions)/self.total_number
        print(f"Loaded {len(self.actions)} actions.")

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.actions[idx], self.cum_rewards[idx]

In [17]:
batch_size = 64
data_path = f"../data/"
weight_path = "models/model_weights_pretrained.pth"
num_epoch_per_run = 1

# optimizer = optim.SGD(net.parameters(), momentum=0.9, lr=0.001)
net = AgentNet()
#net.load_state_dict(torch.load(weight_path))
optimizer = optim.AdamW(net.parameters(), lr=0.001)

In [18]:
optimizer = optim.AdamW(net.parameters(), lr=0.001)
batch_size = 128


In [4]:
EVENT_HORIZON = {
    MOVED_LEFT: 1,
    MOVED_RIGHT: 1,
    MOVED_UP: 1,
    MOVED_DOWN: 1,
    WAITED: 1,
    INVALID_ACTION: 1,
    BOMB_DROPPED: 1,
    BOMB_EXPLODED: 1,
    CRATE_DESTROYED: 6,
    COIN_FOUND: 1,
    COIN_COLLECTED: 5,
    KILLED_OPPONENT: 6,
    KILLED_SELF: 5,
    GOT_KILLED: 6,
    OPPONENT_ELIMINATED: 4,
    SURVIVED_ROUND: 100
}


REWARDS = {
    MOVED_LEFT: 5, #1
    MOVED_RIGHT: 5, #1
    MOVED_UP: 5,   #1
    MOVED_DOWN: 5,  #1
    WAITED: 5,  #1
    INVALID_ACTION: -120, #-7
    BOMB_DROPPED: 5,  #1
    BOMB_EXPLODED: 0,
    CRATE_DESTROYED: 120,
    COIN_FOUND: 0,
    COIN_COLLECTED: 120,
    KILLED_OPPONENT: 60,
    KILLED_SELF: -120, #-12
    GOT_KILLED: -120,
    OPPONENT_ELIMINATED: 5,
    SURVIVED_ROUND: 200 #20
}

In [20]:
for run in range(10):
    # os.system("rm -rf ../rule_based_agent/data")
    os.system("rm -rf ../data")
    os.system("cd ../..; python main.py play --no-gui --agents rule_based_agent rule_based_agent rule_based_agent rule_based_agent --train 4 --n-rounds 40 --scenario loot-crate")

    trainset = BombermanDataset(data_path)
    N = len(trainset)
    print(f"{N} actions in the training set:")
    for i in range(6):
        n = np.sum([t == i for t in trainset.actions])
        print(f"{ACTION_MAP_INV[i]}: {n/N*100:.2f}% ({n})")
    # sampler = BatchSampler(WeightedRandomSampler(trainset.weights, len(trainset), replacement=True,), batch_size, False)
    # trainloader = torch.utils.data.DataLoader(trainset, batch_sampler=sampler)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epoch_per_run):
        running_loss = 0.0
        running = 0
        for i, batch in enumerate(trainloader, 0):
        # for i, batch in enumerate(sampler, 0):
            print(i, end="\r")
            # get the inputs; data is a list of [inputs, labels]
            (channels, features), actions, rewards = batch

            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(channels, features)
            # loss = criterion(outputs, labels)
            log_logits = F.log_softmax(outputs, dim=-1)
            log_probs = log_logits.gather(1, actions[:,np.newaxis])
            
            loss = torch.mean(-log_probs * rewards)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            running += 1
            if i % 200 == 199:    # print every 2000 mini-batches
                print(f'[{run + 1}, {epoch + 1}, {i + 1:5d}] loss: {running_loss / running:.3f}')
                torch.save(net.state_dict(), weight_path)
                running_loss = 0.0
                running = 0
                #print(log_probs)
                #print(rewards)
                print(i*batch_size/N)
            if i*batch_size/N > 0.15:
                break
        if running > 0:
            print(f'[{run + 1}, {epoch + 1}] loss: {running_loss / running:.3f}')
            
    torch.save(net.state_dict(), weight_path)

print('Finished Training') 

100%|██████████| 40/40 [01:02<00:00,  1.57s/it]


rule_based_agent_0    511
rule_based_agent_1    478
rule_based_agent_2    524
rule_based_agent_3    441
Loaded 528784 actions.
528784 actions in the training set:
UP: 19.59% (103597)
DOWN: 25.18% (133160)
LEFT: 19.49% (103045)
RIGHT: 25.55% (135121)
BOMB: 8.60% (45464)
WAIT: 1.59% (8397)
[1, 1,   200] loss: 271.523
0.04817089775787467
[1, 1,   400] loss: 139.790
0.09658386032860299
[1, 1,   600] loss: 118.204
0.1449968228993313
[1, 1] loss: 101.384


100%|██████████| 40/40 [00:55<00:00,  1.39s/it]


rule_based_agent_0    496
rule_based_agent_1    452
rule_based_agent_2    465
rule_based_agent_3    500
Loaded 510991 actions.
510991 actions in the training set:
UP: 19.38% (99032)
DOWN: 25.37% (129626)
LEFT: 19.66% (100450)
RIGHT: 25.12% (128360)
BOMB: 8.68% (44347)
WAIT: 1.80% (9176)
[2, 1,   200] loss: 118.307
0.04984823607460797
[2, 1,   400] loss: 103.313
0.09994696579783205
[2, 1,   600] loss: 93.825
0.15004569552105615


100%|██████████| 40/40 [01:21<00:00,  2.03s/it]


...

rule_based_agent_0    486
rule_based_agent_1    449
rule_based_agent_2    517
rule_based_agent_3    467
Loaded 514745 actions.
514745 actions in the training set:
UP: 19.56% (100665)
DOWN: 24.95% (128407)
LEFT: 19.82% (102026)
RIGHT: 25.38% (130651)
BOMB: 8.68% (44685)
WAIT: 1.61% (8311)
[8, 1,   200] loss: 95.278
0.049484696305937895
[8, 1,   400] loss: 78.088
0.09921805942748352
[8, 1,   600] loss: 74.317
0.14895142254902913
[8, 1] loss: 70.503


100%|██████████| 40/40 [00:55<00:00,  1.39s/it]


rule_based_agent_0    544
rule_based_agent_1    478
rule_based_agent_2    410
rule_based_agent_3    468
Loaded 513918 actions.
513918 actions in the training set:
UP: 19.56% (100499)
DOWN: 25.49% (131002)
LEFT: 19.46% (99996)
RIGHT: 25.32% (130138)
BOMB: 8.53% (43848)
WAIT: 1.64% (8435)
[9, 1,   200] loss: 90.847
0.04956432738296771
[9, 1,   400] loss: 75.577
0.09937772173770913
[9, 1,   600] loss: 67.661
0.14919111609245056
[9, 1] loss: 46.900


100%|██████████| 40/40 [00:54<00:00,  1.37s/it]


rule_based_agent_0    501
rule_based_agent_1    517
rule_based_agent_2    522
rule_based_agent_3    440
Loaded 514822 actions.
514822 actions in the training set:
UP: 19.57% (100732)
DOWN: 24.95% (128455)
LEFT: 19.59% (100868)
RIGHT: 25.62% (131877)
BOMB: 8.60% (44295)
WAIT: 1.67% (8595)
[10, 1,   200] loss: 92.429
0.04947729506509046
[10, 1,   400] loss: 75.591
0.09920321975362359
[10, 1,   600] loss: 66.345
0.1489291444421567
[10, 1] loss: 70.222
Finished Training


In [21]:
torch.save(net.state_dict(), "models/model_weights_pre")

In [22]:
optimizer = optim.AdamW(net.parameters(), lr=0.0001)
batch_size = 256


In [23]:
for run in range(10):
    # os.system("rm -rf ../rule_based_agent/data")
    os.system("rm -rf ../data")
    os.system("cd ../..; python main.py play --no-gui --agents rule_based_agent rule_based_agent rule_based_agent rule_based_agent --train 4 --n-rounds 50 --scenario loot-crate")

    trainset = BombermanDataset(data_path)
    N = len(trainset)
    print(f"{N} actions in the training set:")
    for i in range(6):
        n = np.sum([t == i for t in trainset.actions])
        print(f"{ACTION_MAP_INV[i]}: {n/N*100:.2f}% ({n})")
    # sampler = BatchSampler(WeightedRandomSampler(trainset.weights, len(trainset), replacement=True,), batch_size, False)
    # trainloader = torch.utils.data.DataLoader(trainset, batch_sampler=sampler)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epoch_per_run):
        running_loss = 0.0
        running = 0
        for i, batch in enumerate(trainloader, 0):
        # for i, batch in enumerate(sampler, 0):
            print(i, end="\r")
            # get the inputs; data is a list of [inputs, labels]
            (channels, features), actions, rewards = batch

            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(channels, features)
            # loss = criterion(outputs, labels)
            log_logits = F.log_softmax(outputs, dim=-1)
            log_probs = log_logits.gather(1, actions[:,np.newaxis])
            
            loss = torch.mean(-log_probs * rewards)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            running += 1
            if i % 200 == 199:    # print every 2000 mini-batches
                print(f'[{run + 1}, {epoch + 1}, {i + 1:5d}] loss: {running_loss / running:.3f}')
                torch.save(net.state_dict(), weight_path)
                running_loss = 0.0
                running = 0
                #print(log_probs)
                #print(rewards)
                print(i*batch_size/N)
            if i*batch_size/N > 0.25:
                break
        if running > 0:
            print(f'[{run + 1}, {epoch + 1}] loss: {running_loss / running:.3f}')
            
    torch.save(net.state_dict(), weight_path)

print('Finished Training') 

100%|██████████| 50/50 [01:14<00:00,  1.49s/it]


rule_based_agent_0    603
rule_based_agent_1    586
rule_based_agent_2    599
rule_based_agent_3    588
Loaded 822204 actions.
822204 actions in the training set:
UP: 19.61% (161249)
DOWN: 25.23% (207447)
LEFT: 19.47% (160121)
RIGHT: 25.35% (208467)
BOMB: 8.63% (70983)
WAIT: 1.70% (13937)
[1, 1,   200] loss: 91.364
0.0619602920929599
[1, 1,   400] loss: 81.923
0.12423194243764321
[1, 1,   600] loss: 77.439
0.18650359278232653
[1, 1,   800] loss: 71.511
0.24877524312700985
[1, 1] loss: 72.563


100%|██████████| 50/50 [01:24<00:00,  1.70s/it]


rule_based_agent_0    578
rule_based_agent_1    541
rule_based_agent_2    606
rule_based_agent_3    654
Loaded 847121 actions.
847121 actions in the training set:
UP: 19.25% (163068)
DOWN: 24.86% (210609)
LEFT: 20.00% (169398)
RIGHT: 25.75% (218125)
BOMB: 8.55% (72397)
WAIT: 1.60% (13524)
[2, 1,   200] loss: 92.761
0.060137807940069954
[2, 1,   400] loss: 85.659
0.12057781592003976
[2, 1,   600] loss: 79.833
0.18101782390000956
[2, 1,   800] loss: 73.716
0.24145783187997938
[2, 1] loss: 72.354


100%|██████████| 50/50 [01:13<00:00,  1.47s/it]


...

rule_based_agent_0    587
rule_based_agent_1    554
rule_based_agent_2    631
rule_based_agent_3    622
Loaded 806193 actions.
806193 actions in the training set:
UP: 19.45% (156790)
DOWN: 24.97% (201276)
LEFT: 19.68% (158657)
RIGHT: 25.56% (206035)
BOMB: 8.65% (69774)
WAIT: 1.69% (13661)
[8, 1,   200] loss: 84.761
0.06319082403345104
[8, 1,   400] loss: 74.705
0.12669918989621592
[8, 1,   600] loss: 67.596
0.1902075557589808
[8, 1] loss: 62.399


100%|██████████| 50/50 [01:20<00:00,  1.62s/it]


rule_based_agent_0    585
rule_based_agent_1    620
rule_based_agent_2    542
rule_based_agent_3    646
Loaded 846507 actions.
846507 actions in the training set:
UP: 19.62% (166055)
DOWN: 25.47% (215599)
LEFT: 19.61% (165966)
RIGHT: 25.07% (212243)
BOMB: 8.57% (72534)
WAIT: 1.67% (14110)
[9, 1,   200] loss: 87.836
0.06018142791494931
[9, 1,   400] loss: 76.854
0.12066527506565214
[9, 1,   600] loss: 70.995
0.18114912221635499
[9, 1,   800] loss: 65.990
0.2416329693670578
[9, 1] loss: 63.369


100%|██████████| 50/50 [01:23<00:00,  1.67s/it]


rule_based_agent_0    550
rule_based_agent_1    601
rule_based_agent_2    640
rule_based_agent_3    678
Loaded 821429 actions.
821429 actions in the training set:
UP: 19.63% (161236)
DOWN: 25.13% (206458)
LEFT: 19.76% (162351)
RIGHT: 25.36% (208325)
BOMB: 8.54% (70137)
WAIT: 1.57% (12922)
[10, 1,   200] loss: 83.702
0.06201875025108682
[10, 1,   400] loss: 75.458
0.12434915251348565
[10, 1,   600] loss: 68.025
0.18667955477588447
[10, 1,   800] loss: 61.395
0.2490099570382833
[10, 1] loss: 57.100
Finished Training


In [24]:
torch.save(net.state_dict(), "models/model_weights_fine")

In [9]:
data_path = f"../data/"
weight_path = "models/model_weights.pth"
num_epoch_per_run = 1

# optimizer = optim.SGD(net.parameters(), momentum=0.9, lr=0.001)
net = AgentNet()
net.load_state_dict(torch.load("models/model_weights_fine"))


<All keys matched successfully>

In [10]:
optimizer = optim.AdamW(net.parameters(), lr=0.00001)
batch_size = 256


In [11]:
for run in range(20):
    # os.system("rm -rf ../rule_based_agent/data")
    os.system("rm -rf ../data")
    os.system("cd ../..; python main.py play --no-gui --agents lfov_agent rule_based_agent rule_based_agent rule_based_agent --train 4 --n-rounds 60 --scenario loot-crate --save-winner-game True")

    trainset = BombermanDataset(data_path)
    N = len(trainset)
    print(f"{N} actions in the training set:")
    for i in range(6):
        n = np.sum([t == i for t in trainset.actions])
        print(f"{ACTION_MAP_INV[i]}: {n/N*100:.2f}% ({n})")
    # sampler = BatchSampler(WeightedRandomSampler(trainset.weights, len(trainset), replacement=True,), batch_size, False)
    # trainloader = torch.utils.data.DataLoader(trainset, batch_sampler=sampler)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epoch_per_run):
        running_loss = 0.0
        running = 0
        for i, batch in enumerate(trainloader, 0):
        # for i, batch in enumerate(sampler, 0):
            print(i, end="\r")
            # get the inputs; data is a list of [inputs, labels]
            (channels, features), actions, rewards = batch

            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(channels, features)
            # loss = criterion(outputs, labels)
            log_logits = F.log_softmax(outputs, dim=-1)
            log_probs = log_logits.gather(1, actions[:,np.newaxis])
            
            loss = torch.mean(-log_probs * rewards)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            running += 1
            if i % 200 == 199:    # print every 2000 mini-batches
                print(f'[{run + 1}, {epoch + 1}, {i + 1:5d}] loss: {running_loss / running:.3f}')
                torch.save(net.state_dict(), weight_path)
                running_loss = 0.0
                running = 0
                #print(log_probs)
                #print(rewards)
                print(i*batch_size/N)
            if i*batch_size/N > 0.5:
                break
        if running > 0:
            print(f'[{run + 1}, {epoch + 1}] loss: {running_loss / running:.3f}')
            
    torch.save(net.state_dict(), weight_path)

print('Finished Training') 

100%|██████████| 60/60 [01:28<00:00,  1.47s/it]


lfov_agent    690
rule_based_agent_0    767
rule_based_agent_1    785
rule_based_agent_2    686
Loaded 296379 actions.
296379 actions in the training set:
UP: 19.33% (57281)
DOWN: 25.83% (76543)
LEFT: 19.53% (57892)
RIGHT: 25.42% (75331)
BOMB: 8.62% (25541)
WAIT: 1.28% (3791)
[1, 1,   200] loss: 93.594
0.17188802175592738
[1, 1,   400] loss: 88.953
0.34463980241515085
[1, 1] loss: 84.614


100%|██████████| 60/60 [01:26<00:00,  1.44s/it]


lfov_agent    648
rule_based_agent_0    698
rule_based_agent_1    720
rule_based_agent_2    861
Loaded 300310 actions.
300310 actions in the training set:
UP: 19.39% (58231)
DOWN: 25.46% (76446)
LEFT: 19.64% (58991)
RIGHT: 25.81% (77515)
BOMB: 8.38% (25161)
WAIT: 1.32% (3966)
[2, 1,   200] loss: 83.185
0.16963804069128569
[2, 1,   400] loss: 82.310
0.3401285338483567
[2, 1] loss: 80.394


100%|██████████| 60/60 [01:29<00:00,  1.48s/it]


...

lfov_agent    668
rule_based_agent_0    785
rule_based_agent_1    688
rule_based_agent_2    702
Loaded 284315 actions.
284315 actions in the training set:
UP: 19.63% (55813)
DOWN: 25.98% (73853)
LEFT: 19.15% (54452)
RIGHT: 25.51% (72520)
BOMB: 8.37% (23789)
WAIT: 1.37% (3888)
[18, 1,   200] loss: 78.883
0.17918154159998592
[18, 1,   400] loss: 76.349
0.3592634929567557
[18, 1] loss: 75.507


100%|██████████| 60/60 [01:35<00:00,  1.60s/it]


lfov_agent    670
rule_based_agent_0    667
rule_based_agent_1    726
rule_based_agent_2    778
Loaded 292050 actions.
292050 actions in the training set:
UP: 19.28% (56307)
DOWN: 26.11% (76243)
LEFT: 19.39% (56625)
RIGHT: 25.64% (74882)
BOMB: 8.35% (24377)
WAIT: 1.24% (3616)
[19, 1,   200] loss: 78.297
0.17443588426639275
[19, 1,   400] loss: 77.295
0.3497483307652799
[19, 1] loss: 77.027


100%|██████████| 60/60 [01:26<00:00,  1.45s/it]


lfov_agent    548
rule_based_agent_0    750
rule_based_agent_1    719
rule_based_agent_2    818
Loaded 291567 actions.
291567 actions in the training set:
UP: 19.34% (56386)
DOWN: 25.23% (73551)
LEFT: 19.92% (58072)
RIGHT: 25.85% (75384)
BOMB: 8.43% (24581)
WAIT: 1.23% (3593)
[20, 1,   200] loss: 76.469
0.17472484883405873
[20, 1,   400] loss: 75.179
0.3503277119838665
[20, 1] loss: 74.905
Finished Training


In [12]:
torch.save(net.state_dict(), "models/model_weights_win.pth")


#weight_path_ref = "models/model_weights_pretrained_ref.pth"
#torch.save(net.state_dict(),weight_path_ref)

In [72]:
batch_size = 64
data_path = f"../data/"
weight_path = "models/model_weights.pth"
pretrained_weight_path = "models/model_weights_pretrained.pth"
num_epoch_per_run = 1

In [73]:
# pretrained_net = AgentNet()
# pretrained_net.load_state_dict(torch.load(pretrained_weight_path))
# torch.save(pretrained_net.state_dict(), weigth_path)

In [74]:
net = AgentNet()
pretrained_net = AgentNet()
pretrained_net.load_state_dict(torch.load(pretrained_weight_path))
net.cnn.load_state_dict(pretrained_net.cnn.state_dict())
torch.save(net.state_dict(), weight_path)
optimizer = optim.AdamW(net.mlp.parameters(), lr=0.001)

In [75]:
optimizer = optim.AdamW(net.mlp.parameters(), lr=0.001)

EVENT_HORIZON = {
    MOVED_LEFT: 1,
    MOVED_RIGHT: 1,
    MOVED_UP: 1,
    MOVED_DOWN: 1,
    WAITED: 1,
    INVALID_ACTION: 1,
    BOMB_DROPPED: 1,
    BOMB_EXPLODED: 1,
    CRATE_DESTROYED: 5,
    COIN_FOUND: 3,
    COIN_COLLECTED: 8,
    KILLED_OPPONENT: 8,
    KILLED_SELF: 4,
    GOT_KILLED: 3,
    OPPONENT_ELIMINATED: 7,
    SURVIVED_ROUND: 5
}


REWARDS = {
    MOVED_LEFT: -0.5,  #-0.2
    MOVED_RIGHT: -0.5, #-0.2
    MOVED_UP: -0.5,  #-0.2
    MOVED_DOWN: -0.5,  #-0.2    
    WAITED: 4,
    INVALID_ACTION: -2,
    BOMB_DROPPED: 0,
    BOMB_EXPLODED: 1,
    CRATE_DESTROYED: 2,
    COIN_FOUND: 0,
    COIN_COLLECTED: 6,
    KILLED_OPPONENT: 1,
    KILLED_SELF: -10,
    GOT_KILLED: -1,
    OPPONENT_ELIMINATED: 2,
    SURVIVED_ROUND: 9
}

In [76]:
action_probs = np.zeros((6,500))


In [79]:
for run in range(10):
    # os.system("rm -rf ../rule_based_agent/data")
    os.system("rm -rf ../data")
    #os.system("cd ../..; python main.py play --no-gui --agents basic_agent basic_agent basic_agent basic_agent --train 4 --n-rounds 30 --scenario loot-crate")
    os.system("cd ../..; python main.py play --no-gui --agents basic_agent basic_agent basic_agent basic_agent  --train 4 --n-rounds 30 --scenario loot-crate")

    # os.system("cd ../..; python main.py play --no-gui --agents rule_based_agent rule_based_agent rule_based_agent rule_based_agent --train 4 --n-rounds 10 --scenario classic")

    trainset = BombermanDataset(data_path)
    N = len(trainset)
    print(f"{N} actions in the training set:")
    for i in range(6):
        n = np.sum([t == i for t in trainset.actions])
        print(f"{ACTION_MAP_INV[i]}: {n/N*100:.2f}% ({n})")
        action_probs[i,run] = n/N
    
    
    # sampler = BatchSampler(WeightedRandomSampler(trainset.weights, len(trainset), replacement=True,), batch_size, False)
    # trainloader = torch.utils.data.DataLoader(trainset, batch_sampler=sampler)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epoch_per_run):
        running_loss = 0.0
        running = 0
        for i, batch in enumerate(trainloader, 0):
        # for i, batch in enumerate(sampler, 0):
            print(i, end="\r")
            # get the inputs; data is a list of [inputs, labels]
            (channels, features), actions, rewards = batch

            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(channels, features)
            # loss = criterion(outputs, labels)
            log_logits = F.log_softmax(outputs, dim=-1)
            log_probs = log_logits.gather(1, actions[:,np.newaxis])
            """
            if i == 0:
                print(actions, rewards, log_probs)
                print(-log_probs * rewards)
            """
            loss = torch.mean(-log_probs * rewards)
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            running += 1
            if i % 200 == 199:    # print every 2000 mini-batches
                print(f'[{run + 1}, {epoch + 1}, {i + 1:5d}] loss: {running_loss / running:.3f}')
                torch.save(net.state_dict(), weight_path)
                running_loss = 0.0
                running = 0
        if running > 0:
            print(f'[{run + 1}, {epoch + 1}] loss: {running_loss / running:.3f}')
    torch.save(net.state_dict(), weight_path)

print('Finished Training')

100%|██████████| 30/30 [00:58<00:00,  1.94s/it]


Loaded 298683 actions.
298683 actions in the training set:
UP: 15.22% (45452)
DOWN: 34.32% (102520)
LEFT: 16.40% (48982)
RIGHT: 26.48% (79098)
BOMB: 6.45% (19274)
WAIT: 1.12% (3357)
[1, 1,   200] loss: 20.545
[1, 1,   400] loss: 17.363
[1, 1,   600] loss: 15.881
[1, 1,   800] loss: 15.734
[1, 1,  1000] loss: 14.239
[1, 1,  1200] loss: 13.750
[1, 1,  1400] loss: 13.503
[1, 1,  1600] loss: 13.106
[1, 1,  1800] loss: 12.162
[1, 1,  2000] loss: 12.178
[1, 1,  2200] loss: 11.820
[1, 1,  2400] loss: 11.852
[1, 1,  2600] loss: 11.281
[1, 1,  2800] loss: 11.148
[1, 1,  3000] loss: 10.654
[1, 1,  3200] loss: 11.071
[1, 1,  3400] loss: 10.446
[1, 1,  3600] loss: 10.402
[1, 1,  3800] loss: 9.904
[1, 1,  4000] loss: 9.635
[1, 1,  4200] loss: 9.458
[1, 1,  4400] loss: 9.511
[1, 1,  4600] loss: 9.639
[1, 1] loss: 9.649


100%|██████████| 30/30 [00:58<00:00,  1.94s/it]


Loaded 275019 actions.
275019 actions in the training set:
UP: 14.84% (40818)
DOWN: 35.45% (97483)
LEFT: 16.15% (44411)
RIGHT: 26.31% (72357)
BOMB: 6.03% (16597)
WAIT: 1.22% (3353)
[2, 1,   200] loss: 15.642
[2, 1,   400] loss: 12.522
[2, 1,   600] loss: 10.814
[2, 1,   800] loss: 10.197
[2, 1,  1000] loss: 9.779
[2, 1,  1200] loss: 8.961
[2, 1,  1400] loss: 8.457
[2, 1,  1600] loss: 8.409
[2, 1,  1800] loss: 8.007
[2, 1,  2000] loss: 7.834
[2, 1,  2200] loss: 7.641
[2, 1,  2400] loss: 7.661
[2, 1,  2600] loss: 7.253
[2, 1,  2800] loss: 6.938
[2, 1,  3000] loss: 6.859
[2, 1,  3200] loss: 7.091
[2, 1,  3400] loss: 7.851
[2, 1,  3600] loss: 6.596
[2, 1,  3800] loss: 7.056
[2, 1,  4000] loss: 6.786
[2, 1,  4200] loss: 6.336
[2, 1] loss: 5.841


100%|██████████| 30/30 [01:02<00:00,  2.07s/it]


...

100%|██████████| 30/30 [00:55<00:00,  1.83s/it]


Loaded 266770 actions.
266770 actions in the training set:
UP: 15.18% (40502)
DOWN: 41.63% (111067)
LEFT: 13.93% (37161)
RIGHT: 22.58% (60247)
BOMB: 5.43% (14481)
WAIT: 1.24% (3312)
[7, 1,   200] loss: 10.172
[7, 1,   400] loss: 8.596
[7, 1,   600] loss: 7.097
[7, 1,   800] loss: 6.640
[7, 1,  1000] loss: 6.194
[7, 1,  1200] loss: 6.160
[7, 1,  1400] loss: 5.763
[7, 1,  1600] loss: 5.387
[7, 1,  1800] loss: 5.189
[7, 1,  2000] loss: 5.195
[7, 1,  2200] loss: 5.018
[7, 1,  2400] loss: 5.369
[7, 1,  2600] loss: 4.919
[7, 1,  2800] loss: 5.100
[7, 1,  3000] loss: 5.002
[7, 1,  3200] loss: 5.368
[7, 1,  3400] loss: 4.856
[7, 1,  3600] loss: 5.708
[7, 1,  3800] loss: 5.186
[7, 1,  4000] loss: 4.962
[7, 1] loss: 4.603


100%|██████████| 30/30 [00:52<00:00,  1.74s/it]


Loaded 272686 actions.
272686 actions in the training set:
UP: 14.62% (39861)
DOWN: 41.77% (113900)
LEFT: 11.12% (30333)
RIGHT: 26.36% (71867)
BOMB: 4.95% (13500)
WAIT: 1.18% (3225)
[8, 1,   200] loss: 8.618
[8, 1,   400] loss: 7.104
[8, 1,   600] loss: 6.021
[8, 1,   800] loss: 5.576
[8, 1,  1000] loss: 5.072
[8, 1,  1200] loss: 4.652
[8, 1,  1400] loss: 4.711
[8, 1,  1600] loss: 4.287
[8, 1,  1800] loss: 4.299
[8, 1,  2000] loss: 4.500
[8, 1,  2200] loss: 4.388
[8, 1,  2400] loss: 4.180
[8, 1,  2600] loss: 4.070


KeyboardInterrupt: 

In [39]:
a = np.zeros((1,7))
print(a)
print(a[0])
print(a[0,0])


[[0. 0. 0. 0. 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0.]
0.0


In [51]:
a = np.array([1,2,3,4,5,6,7])-1
print(a)
print(a[1:7])

[0 1 2 3 4 5 6]
[1 2 3 4 5 6]



    extensions = np.zeros(4)
    if pos[0]+4 <= 16:             #field, box_map, explosion_map, others_map, coin_map
        if field[pos[0]+4, pos[1]] == 1:
            extensions[0] = 1
        if box_map[pos[0]+4, pos[1]] ==1:
            extensions[0] = 2
        if explosion_map[pos[0]+4, pos[1]] ==1:
            extensions[0] = 3
        if others_map[pos[0]+4, pos[1]] == 1:
            extensions[0] = 4
        if coin_map[pos[0]+4, pos[1]] ==1:
            extensions[0] =5

    if pos[0]-4 >= 0:             #field, box_map, explosion_map, others_map, coin_map
        if field[pos[0]-4, pos[1]] == 1:
            extensions[1] = 1
        if box_map[pos[0]-4, pos[1]] ==1:
            extensions[1] = 2
        if explosion_map[pos[0]+4, pos[1]] ==1:
            extensions[1] = 3
        if others_map[pos[0]-4, pos[1]] == 1:
            extensions[1] = 4
        if coin_map[pos[0]-4, pos[1]] ==1:
            extensions[1] =5

    if pos[1]+4 <= 16:             #field, box_map, explosion_map, others_map, coin_map
        if field[pos[0], pos[1]+4] == 1:
            extensions[2] = 1
        if box_map[pos[0], pos[1]+4] ==1:
            extensions[2] = 2
        if explosion_map[pos[0], pos[1]+4] ==1:
            extensions[2] = 3
        if others_map[pos[0], pos[1]+4] == 1:
            extensions[2] = 4
        if coin_map[pos[0], pos[1]+4] ==1:
            extensions[2] = 5

    if pos[1]-4 >= 0:             #field, box_map, explosion_map, others_map, coin_map
        if field[pos[0], pos[1]-4] == 1:
            extensions[3] = 1
        if box_map[pos[0], pos[1]-4] ==1:
            extensions[3] = 2
        if explosion_map[pos[0], pos[1]-4] ==1:
            extensions[3] = 3
        if others_map[pos[0], pos[1]-4] == 1:
            extensions[3] = 4
        if coin_map[pos[0], pos[1]-4] ==1:
            extensions[3] = 5

